# Creation of a CSF mask
Goal: Avoid False Positives on ventricles

In [1]:
import sys, os
sys.path.insert(0, '../')
import numpy as np
from gsprep.visual_tools.visual import display, idisplay
from gsprep.visual_tools.dataset_visualization import visualize_dataset
from gsprep.visual_tools.dataset_histograms import plot_per_sample_histograms, plot_histogram
from scipy.ndimage.filters import gaussian_filter

In [2]:
dataset_path = '/Users/julian/temp/perfusion_data_sets/rescaled_with_ncct_dataset.npz'
n_subj = 144

raw_images = np.load(dataset_path, allow_pickle=True)['ct_inputs'][:n_subj].astype(np.float64)
raw_labels = np.load(dataset_path, allow_pickle=True)['ct_lesion_GT'][:n_subj].astype(np.uint8)
# raw_labels = np.load(dataset_path, allow_pickle=True)['lesion_GT'][:n_subj].astype(np.float64)
raw_mask = np.load(dataset_path, allow_pickle=True)['brain_masks'][:n_subj]
params = np.load(dataset_path, allow_pickle=True)['params']

n_subj, n_x, n_y, n_z, n_c = raw_images.shape
raw_images.shape

KeyError: 'ct_lesion_GT is not a file in the archive'

In [ ]:
channel = 4
plot_per_sample_histograms(raw_images[..., channel], log=True, title='NCCT', figsize=(10, 100))



In [ ]:
plot_histogram(raw_images[0, ..., channel], alpha=1)


In [ ]:
def gaussian_smoothing(data, kernel_width=5, threeD=False):
    '''
    Smooth a set of n images with a 2D gaussian kernel on their x, y planes iterating through z
    if threeD is set to false; Every plane in z is smoothed independently
    Every channel is smoothed independently
    :param data: images to smooth (n, x, y, z, c)
    :param kernel_width: 2D or 3D kernel width
        Default width is 5 vxl - (stroke dataset: 10mm width), ie. 5mm radius as inspired by
        Campbell Bruce C.V., Christensen Søren, Levi Christopher R., Desmond Patricia M., Donnan Geoffrey A., Davis Stephen M., et al. Cerebral Blood Flow Is the Optimal CT Perfusion Parameter for Assessing Infarct Core. Stroke. 2011 Dec 1;42(12):3435–40.
    :param threeD, default False: exert smoothing in all 3 spatial dimensions and not only 2
    :return: smoothed_data
    '''
    if len(data.shape) != 5:
        raise ValueError('Shape of data to smooth should be (n, x, y, z, c) and not', data.shape)

    sigma = kernel_width / 3
    truncate = ((kernel_width - 1) / 2 - 0.5) / sigma
    smoothed_data = np.empty(data.shape)

    for i in range(data.shape[0]):
        for c in range(data.shape[4]):
            if not threeD:
                for z in range(data.shape[3]):
                    smoothed_data[i, :, :, z, c] = gaussian_filter(data[i, :, :, z, c], kernel_width / 3,
                                                                   truncate=truncate)
            else:
                smoothed_data[i, :, :, :, c] = gaussian_filter(data[i, :, :, :, c], kernel_width/3, truncate=truncate)

    return smoothed_data

In [ ]:
subj_idx = 13
threshold = 20
display(raw_images[subj_idx, ..., channel], mask=raw_images[None, subj_idx, ..., channel, None]<threshold)
display(raw_images[subj_idx, ..., channel], mask=gaussian_smoothing(raw_images[None, subj_idx, ..., channel, None], kernel_width=3)<threshold)
display(raw_images[subj_idx, ..., channel], mask=-1 * raw_mask[subj_idx] + 1)
display(raw_images[subj_idx, ..., channel])
np.mean(raw_images[subj_idx, ..., channel]), np.std(raw_images[subj_idx, ..., channel]), (threshold - np.mean(raw_images[subj_idx, ..., channel])) / np.std(raw_images[subj_idx, ..., channel])

In [ ]:
idisplay(raw_images[subj_idx, ..., channel], mask=raw_mask[subj_idx])

In [ ]:
ncct_with_csf_mask = np.concatenate((raw_images[..., channel, None], raw_images[ ..., channel, None]<20), axis=-1)
visualize_dataset(ncct_with_csf_mask, ['NCCT', 'CSF'], range(n_subj), os.path.dirname(dataset_path))